In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import argparse
import imutils

Imported all required libraries

In [ ]:
img = cv2.imread('/home/dev/picture-data/img.jpeg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray_blur = cv2.GaussianBlur(gray,(3,3),cv2.BORDER_DEFAULT)

Converting image to grayscale and subsequently convert it to grayscale. Then we filter out high frequency components from image by using Gaussian Blur as upcoming Scharr's operator would generate lots of noise.

In [ ]:
destination_depth = cv2.cv.CV_32F if imutils.is_cv2() else cv2.CV_32F
gradient_X = cv2.Sobel(gray, ddepth=destination_depth, dx=1, dy=0, ksize=-1)
gradient_Y = cv2.Sobel(gray, ddepth=destination_depth, dx=0, dy=1, ksize=-1)
gradient = cv2.subtract(gradient_X, gradient_Y)
gradient = cv2.convertScaleAbs(gradient)

We figure out components in the image with higher horizontal derivative than vertical gradient such as in barcode where vertically same color persists and horizontally blacks and whites change rapidly. In case of a 90 degree rotated barcode in image, we can subtract gradient_X from gradient_Y which has been managed in step 5.

ksize = -1 signifies Scharr's operator which approximates derivative using a different kernel than Sobel which gives a more accurate result.

In [ ]:
im_blur = cv2.GaussianBlur(gradient, (3, 3),cv2.BORDER_DEFAULT)
(_, threshold) = cv2.threshold(im_blur, 225, 255, cv2.THRESH_BINARY)
shape = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
enclosure = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, shape)
enclosure = cv2.erode(enclosure, None, iterations = 4)
enclosure = cv2.dilate(enclosure, None, iterations = 4)
contours = cv2.findContours(enclosure.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
sorted_contours = sorted(contours, key = cv2.contourArea, reverse = True)[0]
approx_rectangle = cv2.minAreaRect(sorted_contours)
bounding_box = cv2.cv.BoxPoints(approx_rectangle) if imutils.is_cv2() else cv2.boxPoints(approx_rectangle)
bounding_box = np.int0(bounding_box)
cv2.drawContours(img, [bounding_box], -1, (0, 255, 0), 3)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()